In [ ]:
# start from 7. image production in cookbook



1 create qpb images

In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
# onserver: mos_exspec_all_allfov.py, esas_exspec_all_allfov.sh


import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]
homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        for sregname in ['allfov']:
            regname = f'{srcname}_{sregname}'

            with open(f'mos_exspec_{sregname}.sh', 'w') as f:
                f.write(
f'''
heainit
sasinit21
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt


#### MOS ####
mkdir -p {regname}
mkdir -p {regname}/{regname}/
mkdir -p {regname}/img/
mkdir -p {regname}/logs/
mkdir -p {regname}/diagnose/
''')
                for E_idx in range(len(ELO_set)):

                    ELO = ELO_set[E_idx]
                    EHI = EHI_set[E_idx]
                        
                    f.write(
f"""
mosspectra eventfile={mos1file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=no elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 |tee {regname}/logs/mosspectra_mos1.log 2>&1 
mosback inspecfile={mos1file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" |tee {regname}/logs/mosback_mos1.log 2>&1 
mosspectra eventfile={mos2file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=no elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 |tee {regname}/logs/mosspectra_mos2.log 2>&1 
mosback inspecfile={mos2file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}"  |tee {regname}/logs/mosback_mos2.log 2>&1 

for name in mos1 mos2
do
echo $name
mv $name*-{ELO}-{EHI}* {regname}/img
mv $name*.qdp {regname}/diagnose
mv $name*imt* {regname}/img
mv $name*imspdet* {regname}/img
done

mv mos1*-fovt.pi {regname}/{regname}/mos1S001-obj-{regname}.pi
mv mos1*-bkg.pi {regname}/{regname}/mos1S001-back-{regname}.pi
mv mos1*.arf {regname}/{regname}/mos1S001-{regname}.arf
mv mos1*.rmf {regname}/{regname}/mos1S001-{regname}.rmf

mv mos2*-fovt.pi {regname}/{regname}/mos2S002-obj-{regname}.pi
mv mos2*-bkg.pi {regname}/{regname}/mos2S002-back-{regname}.pi
mv mos2*.arf {regname}/{regname}/mos2S002-{regname}.arf
mv mos2*.rmf {regname}/{regname}/mos2S002-{regname}.rmf
""")



In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
# onserver: pn_exspec_all_allfov.py, esas_exspec_all_allfov.sh

import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        for sregname in ['allfov']:
            regname = f'{srcname}_{sregname}'

            with open(f'pn_exspec_{sregname}.sh', 'w') as f:
                f.write(
f'''

#### PN ####
heainit
sasinit20
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt
export sname=pn
export name={pnname}
cp $sname*-allevc.fits $name-clean.fits
cp $sname*-allevcoot.fits $name-clean-oot.fits
cp $sname*-corevc.fits $name-corn.fits
cp $sname*-corevcoot.fits $name-corn-oot.fits
cp $sname-bkgregtdet.fits $name-bkg_region-sky.fits 
cp $sname-bkgregtsky.fits $name-bkg_region-det.fits 
''')

                for E_idx in range(len(ELO_set)):

                    ELO = ELO_set[E_idx]
                    EHI = EHI_set[E_idx]
                    f.write(
f'''
for PAT in 0 4
do
pn-spectra prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb region=none mask=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnspectra_$PAT.log 2>&1 
pn_back prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnback_$PAT.log 2>&1 

mkdir -p {regname}/img/pn_$PAT
mkdir -p {regname}/diagnose/pn_$PAT

mv pn*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv pn*-obj.pi {regname}/{regname}/pnS003-$PAT-obj-{regname}.pi
mv pn*-obj-oot.pi {regname}/{regname}/pnS003-$PAT-obj-oot-{regname}.pi
mv pn*-back.pi {regname}/{regname}/pnS003-$PAT-back-{regname}.pi
mv pn*.qdp {regname}/diagnose/pn_$PAT
mv pn*imt* {regname}/img/pn_$PAT
mv pn*.arf {regname}/{regname}/pnS003-$PAT-{regname}.arf
mv pn*.rmf {regname}/{regname}/pnS003-$PAT-{regname}.rmf
mv pn*imspdet* {regname}/img/pn_$PAT

done
''')



2 transform bkg img in det coords to sky coords

In [ ]:
# onserver: create_scripts_rot_det2sky_qpb.py, rot_det2sky_qpb_all.sh

def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('rot_det2sky_qpb.sh', 'w')

        elo = 500
        ehi = 7000

        name_dir = {'mos1':'mos1*-allevc', 'mos2':'mos2*-allevc', 'pn':'pn*-allevc'}

        name_lst = []
        prefix_lst = ''
        for file in glob(f'*0*-allevc.fits'):
            prefix = file.split('-')[0]
            name_lst.append(prefix)

        name_lst = reorder_list(name_lst)

        for name in name_lst:
            f.write(f'rotdet2sky intemplate={name}-fovimsky-{elo}-{ehi}.fits inimage={name}-bkgimdet-{elo}-{ehi}.fits outimage={name}-bkgimsky-{elo}-{ehi}.fits withdetxy=false withskyxy=false\n')
        
        f.close()


3 the proton and swcx routines produce output images in detector coordinates; rotdet2sky can be used to rotate these images as well.